In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os


class PlotDrawer:
    def __init__(self):
        self.plot_folder = "plots"
        if not os.path.exists(self.plot_folder):
            os.makedirs(self.plot_folder)

    def draw_plots(self, json):
        data = pd.read_json(json)
        plots_path = []

        data['correct'] = data['gt_corners'] == data['rb_corners']
        grouped_data = data.groupby('correct').size()
        plt.pie(grouped_data, labels=grouped_data.index.astype(str), autopct='%1.1f%%', colors=['green', 'red'])
        plt.title('Ratio of Correct and Incorrect Answers')
        plt.savefig(os.path.join(self.plot_folder, 'ratio_answers.png'))
        plt.close()
        plots_path.append(os.path.join(self.plot_folder, 'ratio_answers.png'))

        deviation_columns = ['mean', 'max', 'min', 'floor_mean', 'floor_max', 'floor_min', 'ceiling_mean', 'ceiling_max', 'ceiling_min']
        for col in deviation_columns:
            plt.figure(figsize=(8, 5))
            plt.hist(data[col], bins=20, edgecolor='black')
            plt.xlabel('Deviation (degrees)')
            plt.ylabel('Frequency')
            plt.title(f'Distribution of {col.capitalize()} Deviation')
            plt.savefig(os.path.join(self.plot_folder, f'{col}_deviation_distribution.png'))
            plt.close()
            plots_path.append(os.path.join(self.plot_folder, f'{col}_deviation_distribution.png'))

        plt.figure(figsize=(8, 6))
        plt.hist(data['ceiling_mean'], bins=10, alpha=0.5, label='Ceiling deviations')
        plt.hist(data['floor_mean'], bins=10, alpha=0.5, label='Floor deviations')
        plt.xlabel('Deviation, degrees')
        plt.ylabel('Frequency')
        plt.title('Distribution of Ceiling and Floor Deviations')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.plot_folder, 'ceiling_and_floor_deviations.png'))
        plt.close()
        plots_path.append(os.path.join(self.plot_folder, 'ceiling_and_floor_deviations.png'))

        plt.figure(figsize=(8, 6))
        plt.scatter(data['ceiling_mean'], data['floor_mean'], color='blue', alpha=0.7)
        plt.xlabel('Ceiling Deviation, degrees')
        plt.ylabel('Floor Deviation, degrees')
        plt.title('Ceiling Deviations vs. Floor Deviations')
        plt.grid(True)
        plt.savefig(os.path.join(self.plot_folder, 'ceiling_vs_floor_deviations.png'))
        plt.close()
        plots_path.append(os.path.join(self.plot_folder, 'ceiling_vs_floor_deviations.png'))

        boxplot_y = ['mean', 'floor_mean', 'ceiling_mean']
        for col in boxplot_y:
            plt.figure(figsize=(10, 6))
            sns.boxplot(x='gt_corners', y=col, data=data)
            plt.xlabel('Number of Corners in Room')
            plt.ylabel(f'{col} Deviation, degrees')
            plt.title(f'Boxplot of {col} Deviations by Number of Corners in Room')
            plt.grid(True)
            plt.savefig(os.path.join(self.plot_folder, f'{col}_deviations_by_corners.png'))
            plt.close()
            plots_path.append(os.path.join(self.plot_folder, f'{col}_deviations_by_corners.png'))

        mean_deviations = data.groupby('gt_corners')[['ceiling_mean', 'floor_mean']].mean()
        unique_corners = mean_deviations.index
        ceiling_means = mean_deviations['ceiling_mean']
        floor_means = mean_deviations['floor_mean']
        plt.figure(figsize=(10, 6))
        plt.plot(unique_corners, ceiling_means, marker='o', color='blue', label='Ceiling Deviations')
        plt.plot(unique_corners, floor_means, marker='o', color='red', label='Floor Deviations')
        plt.xlabel('Number of Corners in Room')
        plt.ylabel('Mean Deviation, degrees')
        plt.title('Mean Ceiling and Floor Deviations by Number of Corners in Room')
        plt.legend()
        plt.grid(True)
        plt.savefig(os.path.join(self.plot_folder, 'mean_ceiling_and_floor_deviations_by_corners.png'))
        plt.close()
        plots_path.append(os.path.join(self.plot_folder, 'mean_ceiling_and_floor_deviations_by_corners.png'))

        return plots_path


plot_paths = PlotDrawer().draw_plots('data.json')
print("Plots saved at:")
for path in plot_paths:
    print(path)